# Data Collection

### College Football Database

In [ ]:
# Import college football api
#!pip install --q cfbd python-dotenv

In [15]:
# Get general dependencies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import time
from bs4 import BeautifulSoup
import requests
import re

In [16]:
# Install API from college football data
import cfbd
from dotenv import dotenv_values

# Load stuff from .env file
env_vars = dotenv_values('.env')

configuration = cfbd.Configuration()
configuration.api_key['Authorization'] = env_vars.get('CFBD_API_KEY')
configuration.api_key_prefix['Authorization'] = env_vars.get('CFBD_API_KEY_PREFIX')

api_config = cfbd.ApiClient(configuration)

### Team Information

In [17]:
# Grab Team API object
teams_api = cfbd.TeamsApi(api_config)


In [18]:
# Get team info
def get_team_info(**kwargs):
  team_info = kwargs['api'].get_fbs_teams()
  return team_info

# Parse the plays into a dataframe
def team_info_to_df(teams): 
  teams_dict = [dict(
    team = t.school,
    city = t.location.city,
    state = t.location.state,
    mascot = t.mascot,
    abbreviation = t.abbreviation,
    team_id = t.id,
    conference = t.conference,
    stadium_capacity = t.location.capacity,
    latitude = t.location.latitude,
    longitude = t.location.longitude,
    logo = t.logos[0]
  ) for t in teams if t.id is not None]
  teams_info_df = pd.DataFrame(teams_dict)
  return teams_info_df

In [19]:
# Grab Team API object
teams_api = cfbd.TeamsApi(api_config)

# Run our functions to get all info we need from that API
team_info_df = team_info_to_df(
    get_team_info(api=teams_api)
)
team_info_df.head()

,team,city,state,mascot,abbreviation,team_id,conference,stadium_capacity,latitude,longitude,logo
0,Air Force,Colorado Springs,CO,Falcons,AFA,2005,Mountain West,46692.0,38.996970,-104.843616,http://a.espncdn.com/i/teamlogos/ncaa/500/2005...
1,Akron,Akron,OH,Zips,AKR,2006,Mid-American,30000.0,41.072553,-81.508341,http://a.espncdn.com/i/teamlogos/ncaa/500/2006...
2,Alabama,Tuscaloosa,AL,Crimson Tide,ALA,333,SEC,101821.0,33.208275,-87.550384,http://a.espncdn.com/i/teamlogos/ncaa/500/333.png
3,Appalachian State,Boone,NC,Mountaineers,APP,2026,Sun Belt,30000.0,36.211427,-81.685428,http://a.espncdn.com/i/teamlogos/ncaa/500/2026...
4,Arizona,Tucson,AZ,Wildcats,ARIZ,12,Pac-12,50782.0,32.228805,-110.948868,http://a.espncdn.com/i/teamlogos/ncaa/500/12.png


In [14]:
team_info_df.to_csv('data/team_info.csv', index=False)

### Team Records Info (2000-Present)

In [6]:
# Get team records
def get_records(**kwargs):
  records = []
  for year in kwargs['years']:
    for team in kwargs['teams']:    
      records += kwargs['api'].get_team_records(year=year, team=team)
    time.sleep(1)  
  return records

# Parse the plays into a dataframe
def records_to_df(records): 
  records_dict = [dict(
    team = r.team,
    team_id = r.team_id,
    year = r.year,
    conference = r.conference,
    games_played = r.total.games,
    expected_wins = r.expected_wins,
    wins = r.total.wins,
    losses = r.total.losses,
    home_wins = r.home_games.wins,
    home_losses = r.home_games.losses,
    away_wins = r.away_games.wins,
    away_losses = r.away_games.losses,
  ) for r in records if r.team is not None]
  records_df = pd.DataFrame(records_dict)
  return records_df 

Note: The CFBD doesn't handle numpy.int64 types. You need to use range() rather than list(np.arange()) because the elements of a list from range will be regular python ints.

In [38]:
type(list(np.arange(2000,2002))[0])

numpy.int64

In [39]:
#games_api.get_team_records(year=2023, team='South Carolina')
years = range(2000,2023+1)
teams = [team for team in team_info_df.team]

records_df = records_to_df(
    get_records(api=games_api, years=years, teams=teams)
)
records_df.head()

,team,team_id,year,conference,games_played,expected_wins,wins,losses,home_wins,home_losses,away_wins,away_losses
0,Air Force,2005,2000,Mountain West,12,0.0,9,3,5,1,3,2
1,Akron,2006,2000,Mid-American,11,0.0,6,5,3,3,3,2
2,Alabama,333,2000,SEC,11,0.0,3,8,3,3,0,5
3,Arizona,12,2000,Pac-10,11,0.0,5,6,2,4,3,2
4,Arizona State,9,2000,Pac-10,12,0.0,6,6,3,3,3,2


In [41]:
records_df.to_csv('data/team_records_by_year.csv', index=False)

In [15]:
test = pd.read_csv('data/team_records_by_year.csv')
test[(test.team == 'South Carolina') & (test.year == 2023)]

,team,team_id,year,conference,games_played,expected_wins,wins,losses,home_wins,home_losses,away_wins,away_losses
2912,South Carolina,2579,2023,SEC,12,5.6,5,7,5,2,0,4


### Team Performance Ratings

In [18]:
ratings_api = cfbd.RatingsApi(api_config)

In [15]:
# Get team records
def get_ratings(**kwargs):
  elos = []
  fpis = []
  #sps = []
  for year in kwargs['years']:
    for team in kwargs['teams']:    
        elos += kwargs['api'].get_elo_ratings(year=year, team=team)
        fpis += kwargs['api'].get_fpi_ratings(year=year, team=team)
        conf_rating += kwargs['api'].get_conference_sp_ratings(year=year, conferenece=team)
  #time.sleep(1)  
  return {'elos': elos, 'fpis': fpis}

# Parse the plays into a dataframe
def ratings_to_df(**kwargs): 
  elos_df = pd.DataFrame([dict(team=r.team, year=r.year, conference=r.conference, elo=r.elo) for r in kwargs['elo'] if r.team is not None])
  fpis_df = pd.DataFrame([dict(team=r.team, year=r.year, conference=r.conference, fpi=r.fpi) for r in kwargs['fpi'] if r.team is not None])
  #sps_df = pd.DataFrame([dict(team=r.team, year=r.year, conference=r.conference, sp=r.sp) for r in kwargs['sp'] if r.team is not None])
  ratings_df = pd.merge(elos_df, fpis_df, on=['team', 'year', 'conference'])
  return ratings_df 

In [81]:
years = range(2000,2023+1)
teams = [team for team in team_info_df.team]
team_ratings_df = ratings_to_df(elo = get_ratings(api=ratings_api, years = years, teams = teams)['elos'],
                           fpi = get_ratings(api=ratings_api, years = years, teams = teams)['fpis'])
team_ratings_df.head()

,team,year,conference,elo,fpi
0,Air Force,2005,Mountain West,1419.0,-2.383
1,Akron,2005,Mid-American,1394.0,-8.552
2,Alabama,2005,SEC,1798.0,14.764
3,Arizona,2005,Pac-10,1480.0,4.534
4,Arizona State,2005,Pac-10,1692.0,14.189


In [82]:
# Combine with conference ratings
def get_conference_ratings(years):
    conferences, ratings, years_list = [], [], []
    for year in years:
        conference_ratings = ratings_api.get_conference_sp_ratings(year=year)
        conferences += [element.conference for element in conference_ratings]
        ratings += [element.rating for element in conference_ratings]
        years_list += [element.year for element in conference_ratings]
    return pd.DataFrame({'conference': conferences, 'conference_rating': ratings, 'year': years_list})

In [83]:
years = range(2000,2023+1)
conference_ratings_df = get_conference_ratings(years)
conference_ratings_df.head()

,conference,conference_rating,year
0,ACC,3.477778,2000
1,Big 12,9.450000,2000
2,Big East,12.175000,2000
3,Big Ten,8.190909,2000
4,Big West,-10.683333,2000


In [84]:
team_conf_ratings_df = pd.merge(team_ratings_df, conference_ratings_df, on=['conference','year'])
team_conf_ratings_df

,team,year,conference,elo,fpi,conference_rating
0,Air Force,2005,Mountain West,1419.0,-2.383,-0.811111
1,Akron,2005,Mid-American,1394.0,-8.552,-10.025000
2,Alabama,2005,SEC,1798.0,14.764,7.625000
3,Arizona,2005,Pac-10,1480.0,4.534,11.090000
4,Arizona State,2005,Pac-10,1692.0,14.189,11.090000
...,...,...,...,...,...,...
2190,Western Kentucky,2023,Conference USA,1449.0,-5.151,-7.888889
2191,Western Michigan,2023,Mid-American,1223.0,-13.543,-11.308333
2192,West Virginia,2023,Big 12,1611.0,5.931,5.514286
2193,Wisconsin,2023,Big Ten,1631.0,6.043,6.078571


In [85]:
team_conf_ratings_df.to_csv('data/team_conference_ratings.csv', index=False)

## Recruiting Data
- Team recruiting rankings and ratings for all teams 2000-2023
- Player rankings for each team all years
    - Get blue chip ratio
    - Usable for recruiting visualizer

#### Team Recruiting Rankings

In [21]:
# Get recruiting api
recruiting_api = cfbd.RecruitingApi(api_config)

In [91]:
# Get team recruiting object and just return everything
team_recruiting = recruiting_api.get_recruiting_teams()

In [98]:
# Traverse the object and turn into DF
points = [entry.points for entry in team_recruiting]
ranks = [entry.rank for entry in team_recruiting]
teams = [entry.team for entry in team_recruiting]
years = [entry.year for entry in team_recruiting]

team_recruiting_df = pd.DataFrame({'team': teams, 'year': years, 'rank': ranks, 'points': points})
team_recruiting_df.head()

,team,year,rank,points
0,Tennessee,2000,1,252.66
1,Florida,2000,2,239.17
2,Florida State,2000,3,225.16
3,Alabama,2000,4,199.31
4,Penn State,2000,5,196.61


In [124]:
team_recruiting_df.to_csv('data/team_recruiting.csv', index=False)

#### Player Recruiting Data

In [23]:
# Get all recruits for each year for Streamlit visualizer and blue chip ratio
player_recruiting = recruiting_api.get_recruiting_players(year=2024)
player_recruiting[0]

{'athlete_id': None,
 'city': 'Opa Locka',
 'committed_to': 'Ohio State',
 'country': 'USA',
 'height': 75.0,
 'hometown_info': {'county_fips': None,
                   'latitude': 25.896733849999997,
                   'longitude': -80.25949640850294},
 'id': 95144,
 'name': 'Jeremiah Smith',
 'position': 'WR',
 'ranking': 1,
 'rating': 0.9997,
 'recruit_type': 'HighSchool',
 'school': 'Chaminade-Madonna Prep',
 'stars': 5,
 'state_province': 'FL',
 'weight': 198,
 'year': 2024}

In [24]:
year_list = range(2000,2024 + 1)

names, years, stars, schools, states, rankings, ratings, latitudes, longitudes, states = [], [], [], [], [], [], [], [], [], []
positions, heights, weights = [], [], []
for year in year_list:
    player_recruiting = recruiting_api.get_recruiting_players(year=year)
    for entry in player_recruiting:
        names.append(entry.name)
        years.append(entry.year)
        stars.append(entry.stars)
        schools.append(entry.committed_to)
        states.append(entry.state_province)
        rankings.append(entry.ranking)
        ratings.append(entry.rating)
        latitudes.append(entry.hometown_info.latitude),
        longitudes.append(entry.hometown_info.longitude)
        positions.append(entry.position)
        heights.append(entry.height)
        weights.append(entry.weight)
player_recruiting_df = pd.DataFrame({'name': names, 'year': years, 'star': stars,
                                     'school': schools, 'state': states, 'ranking': rankings,
                                     'rating': ratings, 'latitude': latitudes, 'longitude': longitudes,
                                     'position': positions, 'height': heights, 'weight': weights})
player_recruiting_df.head()

,name,year,star,school,state,ranking,rating,latitude,longitude,position,height,weight
0,D.J. Williams,2000,5,Miami,CA,1.0,0.9998,37.976852,-122.033562,ILB,74.0,235.0
1,Brock Berlin,2000,5,Florida,LA,2.0,0.9998,32.522183,-93.765194,PRO,74.0,190.0
2,Charles Rogers,2000,5,Michigan State,MI,3.0,0.9988,43.420039,-83.949037,WR,76.0,195.0
3,Travis Johnson,2000,5,Florida State,CA,4.0,0.9982,34.150872,-118.448987,SDE,76.0,265.0
4,Marcus Houston,2000,5,Colorado,CO,5.0,0.9980,39.739236,-104.984862,RB,72.0,208.0


In [25]:
player_recruiting_df.to_csv('data/player_recruiting.csv', index=False)

#### Most Recent Year Recruiting Data
- Source 247 sports composite rankings that aren't on CFBD yet.

In [72]:
url = 'https://247sports.com/Season/2024-Football/CompositeTeamRankings/'

headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}

response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.content, 'html.parser')
table = soup.find_all('li', {'class': 'rankings-page__list-item'})

In [76]:
years, rankings, teams, totals, five_stars, four_stars, avgs, points = [], [], [], [], [], [], [], []
for row in table:
    years.append(2024)
    rankings.append(row.find('div', {'class': 'rank-column'}).find('div', {'class': 'primary'}).text.strip())
    teams.append(row.find('a', {'class': 'rankings-page__name-link'}).text.strip())
    totals.append(int(re.findall(r'\d+', row.find('div', {'class': 'total'}).text)[0]))
    five_stars.append(int(re.findall(r'\s(\d+)', row.find('ul', {'class':'star-commits-list'}).find_all('li')[0].text.strip())[0]))
    four_stars.append(int(re.findall(r'\s(\d+)', row.find('ul', {'class':'star-commits-list'}).find_all('li')[1].text.strip())[0]))
    avgs.append(float(row.find('div', {'class': 'avg'}).text.strip()))
    points.append(float(row.find('div', {'class': 'points'}).text.strip()))

newest_rankings_df = pd.DataFrame({'rank': rankings, 'points': points, 'year': years, 
                                   'team': teams, 'four_stars': four_stars, 'five_stars': five_stars,
                                   'total': totals, 'average_player': avgs, 'points': points})
newest_rankings_df['blue_chip'] = newest_rankings_df['four_stars'] + newest_rankings_df['five_stars']

newest_rankings_df.head()


,rank,points,year,team,four_stars,five_stars,total,average_player,blue_chip
0,1,317.05,2024,Georgia,19,5,28,93.61,24
1,2,310.74,2024,Alabama,17,5,28,93.12,22
2,3,293.20,2024,Oregon,22,0,27,92.19,22
3,4,291.52,2024,Miami,11,3,27,91.00,14
4,5,289.10,2024,Ohio State,14,4,23,92.79,18


In [77]:
newest_rankings_df.to_csv('data/newest_recruiting_rankings.csv', index=False)

### Player Data (only from 2014 - Present)
- Returning players
- Returning production

In [125]:
player_api = cfbd.PlayersApi(api_config)

In [136]:
returning = player_api.get_returning_production(year=2014)

In [137]:
years_list = range(2014, 2023+1)
years, teams, conferences, passing_usages, rushing_usages, usages = [], [], [], [] ,[] ,[] 
for year in years_list:
    returning = player_api.get_returning_production(year=year)
    for row in returning:
        years.append(row.season)
        teams.append(row.team)
        conferences.append(row.conference)
        passing_usages.append(row.passing_usage)
        rushing_usages.append(row.rushing_usage)
        usages.append(row.usage)

returning_df = pd.DataFrame({'year': years, 'team': teams, 'conference': conferences,
                             'passing_usage': passing_usages, 'rushing_usage': rushing_usages, 'usages': usages})
returning_df.head()    

,year,team,conference,passing_usage,rushing_usage,usages
0,2014,Air Force,Mountain West,0.751,0.652,0.684
1,2014,Akron,Mid-American,1.000,0.989,0.952
2,2014,Alabama,SEC,0.078,0.911,0.589
3,2014,Arizona,Pac-12,0.000,0.048,0.142
4,2014,Arizona State,Pac-12,0.990,0.562,0.713


In [138]:
returning_df.to_csv('data/returning_players_2014.csv', index=False)

### Team Stats (Advanced Stats + Season Stats)
- Get season by season stats for teams
- Focus on stats that would help define if returning players were good or not
- NOTE: The farther back in years (such as 2001, or 2002) there are fewer stats available

In [3]:
# Get stats api object
stats_api = cfbd.StatsApi(api_config)

In [19]:
season_stats = stats_api.get_team_season_stats(year=2007)

In [20]:
# Create dataframe of all data
conferences, years, stat_names, stat_values, teams = [], [], [], [], []
year_list = range(2007, 2023+1)
for year in year_list:
    season_stats = stats_api.get_team_season_stats(year=year)
    for entry in season_stats:
        conferences.append(entry.conference)
        years.append(entry.season)
        stat_names.append(entry.stat_name)
        stat_values.append(entry.stat_value)
        teams.append(entry.team)
season_stats_df = pd.DataFrame({'conference': conferences, 'year': years, 
                                'stat_name': stat_names, 'stat_value': stat_values, 'team': teams})
season_stats_df    

,conference,year,stat_name,stat_value,team
0,Big Ten,2007,puntReturns,25,Minnesota
1,Big Ten,2007,totalYards,4616,Northwestern
2,SEC,2007,firstDowns,246,Auburn
3,Sun Belt,2007,fumblesRecovered,6,Louisiana Monroe
4,Big 12,2007,interceptionYards,103,Kansas State
...,...,...,...,...,...
65733,Sun Belt,2023,sacks,20,Coastal Carolina
65734,Big Ten,2023,kickReturns,22,Purdue
65735,FBS Independents,2023,passingTDs,14,UMass
65736,Mid-American,2023,thirdDowns,180,Eastern Michigan


In [23]:
# Turn into df of teams and stats as columns using pivot table
pivot_df = season_stats_df.pivot_table(index=['team', 'year', 'conference'], 
                                    columns='stat_name', values='stat_value').reset_index()

pivot_df.head()

stat_name,team,year,conference,firstDowns,fourthDownConversions,fourthDowns,fumblesLost,fumblesRecovered,games,interceptionTDs,...,puntReturns,rushingAttempts,rushingTDs,rushingYards,sacks,tacklesForLoss,thirdDownConversions,thirdDowns,totalYards,turnovers
0,Air Force,2007,Mountain West,277.0,17.0,25.0,13.0,15.0,13.0,1.0,...,24.0,721.0,36.0,3894.0,NaN,NaN,79.0,188.0,5452.0,18.0
1,Air Force,2008,Mountain West,240.0,13.0,24.0,11.0,21.0,13.0,1.0,...,36.0,777.0,27.0,3470.0,NaN,NaN,87.0,203.0,4538.0,17.0
2,Air Force,2009,Mountain West,261.0,27.0,38.0,9.0,14.0,13.0,3.0,...,33.0,815.0,31.0,3685.0,NaN,NaN,100.0,226.0,4834.0,12.0
3,Air Force,2010,Mountain West,286.0,18.0,30.0,11.0,10.0,13.0,0.0,...,27.0,748.0,41.0,3985.0,NaN,NaN,94.0,186.0,5536.0,17.0
4,Air Force,2011,Mountain West,301.0,26.0,32.0,18.0,14.0,13.0,0.0,...,33.0,723.0,43.0,4092.0,NaN,NaN,85.0,181.0,5913.0,26.0


In [24]:
pivot_df.to_csv('data/season_stats.csv', index=False)

#### Get Advanced Stats
- Predicted points added
- success rate
- explosiveness

In [9]:
def get_advanced_stats_df(team):
    season_stats = stats_api.get_advanced_team_season_stats(team=team)
    df = pd.DataFrame.from_records(s.to_dict() for s in season_stats)
    df['off_drives'] = df['offense'].apply(lambda row: row['drives'])
    df['def_drives'] = df['defense'].apply(lambda row: row['drives'])
    df['off_explode'] = df['offense'].apply(lambda row: row['explosiveness'])
    df['def_explode'] = df['defense'].apply(lambda row: row['explosiveness'])
    df['off_ppa'] = df['offense'].apply(lambda row: row['ppa'])
    df['def_ppa'] = df['defense'].apply(lambda row: row['ppa'])
    df['off_success_rate'] = df['offense'].apply(lambda row: row['success_rate'])
    df['def_success_rate'] = df['defense'].apply(lambda row: row['success_rate'])
    return df

In [13]:
teams_list = list(team_info_df.team)
advanced_stats_df = pd.DataFrame()

for team in teams_list:
    advanced_stats_df = pd.concat([advanced_stats_df, get_advanced_stats_df(team)], ignore_index=True)

advanced_stats_df.head()

,season,team,conference,offense,defense,off_drives,def_drives,off_explode,def_explode,off_ppa,def_ppa,off_success_rate,def_success_rate
0,2001,Air Force,Mountain West,"{'plays': 85, 'drives': 18, 'ppa': -0.22851381...","{'plays': 67, 'drives': 14, 'ppa': 0.051421327...",18,14,0.764806,0.750151,-0.228514,0.051421,0.305882,0.417910
1,2002,Air Force,Mountain West,"{'plays': 75, 'drives': 11, 'ppa': -0.10558718...","{'plays': 60, 'drives': 9, 'ppa': -0.012989655...",11,9,1.349488,0.784216,-0.105587,-0.012990,0.266667,0.433333
2,2003,Air Force,Mountain West,"{'plays': 68, 'drives': 15, 'ppa': -0.00534472...","{'plays': 77, 'drives': 15, 'ppa': -0.03854771...",15,15,0.771551,1.205097,-0.005345,-0.038548,0.382353,0.376623
3,2004,Air Force,Mountain West,"{'plays': 156, 'drives': 27, 'ppa': 0.05003897...","{'plays': 123, 'drives': 28, 'ppa': 0.15689943...",27,28,0.922500,0.837418,0.050039,0.156899,0.384615,0.520325
4,2005,Air Force,Mountain West,"{'plays': 611, 'drives': 108, 'ppa': 0.0591593...","{'plays': 580, 'drives': 102, 'ppa': 0.0235214...",108,102,0.853179,0.982010,0.059159,0.023521,0.417349,0.398276


In [14]:
advanced_stats_df.to_csv('data/advanced_stats_seasons.csv', index=False)

In [68]:
df = pd.DataFrame.from_records(g.to_dict() for g in stats_api.get_team_season_stats(year=2023))
df.head()

,season,team,conference,stat_name,stat_value
0,2023,UT San Antonio,American Athletic,rushingAttempts,516
1,2023,California,Pac-12,thirdDownConversions,74
2,2023,San Diego State,Mountain West,interceptions,10
3,2023,Rice,American Athletic,kickReturns,26
4,2023,Texas,Big 12,interceptionTDs,2


### Draft Stats
- Get draft picks by team to potentially investigate recruiting vs. draft picks

In [13]:
draft_api = cfbd.DraftApi(api_config)

In [14]:
draft = draft_api.get_draft_picks(year=2024)

In [15]:
years, names, teams, conferences, picks, rounds, positions = [], [], [], [] ,[] ,[], []
year_list = range(2000,2024+1)
for year in year_list:
    draft = draft_api.get_draft_picks(year=year)
    for row in draft:
        names.append(row.name)
        years.append(row.year)
        teams.append(row.college_team)
        conferences.append(row.college_conference)
        picks.append(row.pick)
        rounds.append(row.round)
        positions.append(row.position)

draft_df = pd.DataFrame({'name': names, 'team': teams, 'conference': conferences, 'year': years,
                             'pick': picks, 'round': rounds, 'position': positions})
draft_df.head()

,name,team,conference,year,pick,round,position
0,Courtney Brown,Penn State,Big Ten,2000,1,1,Defensive End
1,LaVar Arrington,Penn State,Big Ten,2000,2,1,Linebacker
2,Chris Samuels,Alabama,SEC,2000,3,1,Offensive Tackle
3,Peter Warrick,Florida State,ACC,2000,4,1,Wide Receiver
4,Jamal Lewis,Tennessee,SEC,2000,5,1,Running Back


In [16]:
draft_df.to_csv('data/draft_info.csv', index=False)

### Coaching Information
- Goal: Get coaching win percentage up to but not including each season.
- Issue: 2023 data is INCOMPLETE. Only tracks through half (6 games) of 2023 season

In [3]:
coaches_api = cfbd.CoachesApi(api_config)
coaches = coaches_api.get_coaches()

In [4]:
years, names, teams, games, wins, losses = [], [], [], [], [], []
for coach in coaches:
    for season in coach.seasons:
        # Create entries for every coach for every season
        names.append(coach.first_name + ' ' + coach.last_name)
        teams.append(season.school)
        years.append(season.year)
        games.append(season.games)
        wins.append(season.wins)
        losses.append(season.losses)

coaches_df = pd.DataFrame({'name': names, 'team': teams, 'year': years, 'games': games, 'wins': wins, 'losses': losses})
coaches_df.head()

,name,team,year,games,wins,losses
0,Eli Abbott,Alabama,1902,8,4,4
1,Earl Abell,Colgate,1928,9,6,3
2,Earl Abell,Virginia,1929,9,4,3
3,Earl Abell,Virginia,1930,10,4,6
4,Earl Able,Mississippi State,1923,9,5,2


In [5]:
coaches_df[coaches_df.name == 'Lincoln Riley']

,name,team,year,games,wins,losses
8910,Lincoln Riley,Oklahoma,2017,14,12,2
8911,Lincoln Riley,Oklahoma,2018,14,12,2
8912,Lincoln Riley,Oklahoma,2019,14,12,2
8913,Lincoln Riley,Oklahoma,2020,11,9,2
8914,Lincoln Riley,Oklahoma,2021,12,10,2
8915,Lincoln Riley,USC,2022,14,11,3
8916,Lincoln Riley,USC,2023,3,3,0


In [6]:
coaches_df.to_csv('data/coaches.csv', index=False)

### Team Stats Game by Game

### Getting Predicted Win Totals Farther Back

## On3 NIL Valuations
https://www.on3.com/nil/news/about-on3-nil-valuation-per-post-value/

In [95]:
# Function to format followers into integers
def values_to_int(string):
    if string[0] == '$':
        string = string[1::]
    if 'K' in string:
        number = float(string[:-1])  # Convert the string to float, excluding the 'K' suffix
        return int(number * 1000)    # Multiply the number by 1000 and convert it to an integer
    elif 'M' in string:
        number = float(string[:-1])
        return int(number * 1000000)
    else:
        return int(float(string))     # Convert the string to float and then to an integer

In [11]:
#scrape_NIL_100('https://www.on3.com/nil/rankings/player/nil-100/')
url = 'https://www.on3.com/nil/rankings/player/college/football/'
response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')
table = soup.find_all('div', {'class': 'NilPlayerRankingItem_itemContainer___Uo0_'})

In [164]:
names, ranks, schools, positions, followers, valuations = [], [], [], [], [], []
for row in table:
    
    names.append(row.find('div', {'class': 'NilPlayerRankingItem_nameYearContainer__kVMqH'}).text)
    
    ranks.append(row.find('span', {'class': "MuiTypography-root MuiTypography-body1 NilPlayerRankingItem_playerRank__NQmBq css-z52hnt"}).text)
    
    school = row.find('div', {'class': 'NilPlayerRankingItem_statusItem__gikz_'})
    if school:
        schools.append(row.find('div', {'class': 'NilPlayerRankingItem_statusItem__gikz_'}).find('img')['title'])
    else:
        schools.append(None)    
    
    positions.append(row.find('span', {'class': "MuiTypography-root MuiTypography-body1 NilPlayerRankingItem_position__WIvtI css-z52hnt"}).text)
    
    followers.append(values_to_int(row.find('p', {'class': "MuiTypography-root MuiTypography-body1 NilPlayerRankingItem_followersNumber__xG05J css-z52hnt"}).text))
    
    valuation_container = row.find('div', {'class': 'NilPlayerRankingItem_valuationContainer__nV9Sj'})
    valuation = valuation_container.find('p',{'class': 'MuiTypography-root MuiTypography-body1 NilPlayerRankingItem_valuationCurrency___Pa_U css-z52hnt'})
    if valuation:
        valuations.append(values_to_int(valuation.text))
    else:
        valuations.append(None)    

nil_df = pd.DataFrame({'name': names, 'rank': ranks, 'school': schools,
                       'position': positions, 'follwers': followers, 'valuation': valuations})
nil_df.head() 


,name,rank,school,position,follwers,valuation
0,Shedeur Sanders,1,colorado buffaloes,QB,2500000,4600000.0
1,Travis Hunter,2,colorado buffaloes,CB,2300000,2700000.0
2,Arch Manning,3,texas longhorns,QB,301000,2400000.0
3,Quinn Ewers,4,texas longhorns,QB,265000,1900000.0
4,Jalen Milroe,5,alabama crimson tide,QB,223000,1600000.0


In [167]:
nil_df.to_csv('data/nil_data.csv', index=False)